In [1]:
# Import dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# import scipy.stats as stats
# from scipy.stats import linregress
# from scipy.stats.stats import pearsonr
import warnings
%matplotlib inline

In [2]:
# Read-in data as Excel file
vix_data_file = "C:/Users/blake/Desktop/spx_vix_clean.xlsx"

In [3]:
#Convert Excel file to Dataframe
stock_data_df = pd.read_excel(vix_data_file,
sheet_name=0,
header=1,
index_col=False,
keep_default_na=False
)

In [4]:
stock_data_df.head()

,Date,SPX,VIX_1,VIX_2,VIX_3,VIX_4,VIX_5,VIX_6,VIX_7,VIX_8,stdev,1/stdev,SPX_VOLUME,SPX_5day,SPX_tom,A_Delta
0,2009-12-31,1115.10,22.95,25.80,26.00,26.30,26.35,26.30,26.40,26.35,1.172737,0.852706,636905,NaN,1132.98,17.88
1,2010-01-01,1132.98,22.10,24.85,25.25,25.75,25.75,25.75,25.85,25.95,1.287977,0.776412,636905,NaN,1132.98,0.00
2,2010-01-04,1132.98,22.10,24.85,25.25,25.75,25.75,25.75,25.85,25.95,1.287977,0.776412,1282633,NaN,1136.52,3.54
3,2010-01-05,1136.52,21.55,24.45,24.85,25.45,25.50,25.50,25.55,25.70,1.386655,0.721160,1368386,NaN,1137.14,0.62
4,2010-01-06,1137.14,20.80,23.55,24.20,24.95,25.05,25.05,25.10,25.30,1.511385,0.661645,1252015,1130.944,1141.69,4.55


In [5]:
#Create new Dataframe, set 'Date' as index, drop column named 'Date' 
time_indexed_stock_df = stock_data_df.set_index(stock_data_df['Date'])
time_indexed_stock_df = time_indexed_stock_df.drop(['Date'], axis=1)
time_indexed_stock_df.head()

,SPX,VIX_1,VIX_2,VIX_3,VIX_4,VIX_5,VIX_6,VIX_7,VIX_8,stdev,1/stdev,SPX_VOLUME,SPX_5day,SPX_tom,A_Delta
Date,,,,,,,,,,,,,,,
2009-12-31,1115.10,22.95,25.80,26.00,26.30,26.35,26.30,26.40,26.35,1.172737,0.852706,636905,NaN,1132.98,17.88
2010-01-01,1132.98,22.10,24.85,25.25,25.75,25.75,25.75,25.85,25.95,1.287977,0.776412,636905,NaN,1132.98,0.00
2010-01-04,1132.98,22.10,24.85,25.25,25.75,25.75,25.75,25.85,25.95,1.287977,0.776412,1282633,NaN,1136.52,3.54
2010-01-05,1136.52,21.55,24.45,24.85,25.45,25.50,25.50,25.55,25.70,1.386655,0.721160,1368386,NaN,1137.14,0.62
2010-01-06,1137.14,20.80,23.55,24.20,24.95,25.05,25.05,25.10,25.30,1.511385,0.661645,1252015,1130.944,1141.69,4.55


In [6]:
#Drop NaN rows
spx_5day_df = time_indexed_stock_df.dropna()
spx_5day_df.head()

,SPX,VIX_1,VIX_2,VIX_3,VIX_4,VIX_5,VIX_6,VIX_7,VIX_8,stdev,1/stdev,SPX_VOLUME,SPX_5day,SPX_tom,A_Delta
Date,,,,,,,,,,,,,,,
2010-01-06,1137.14,20.80,23.55,24.20,24.95,25.05,25.05,25.10,25.30,1.511385,0.661645,1252015,1130.944,1141.69,4.55
2010-01-07,1141.69,20.45,23.15,23.85,24.75,24.85,24.85,24.90,25.10,1.575425,0.634749,1553963,1136.262,1144.98,3.29
2010-01-08,1144.98,19.90,22.55,23.40,24.40,24.65,24.60,24.75,24.90,1.714317,0.583323,1508175,1138.662,1146.98,2.00
2010-01-11,1146.98,19.50,22.25,23.25,24.15,24.30,24.40,24.50,24.65,1.760479,0.568027,1444997,1141.462,1136.22,-10.76
2010-01-12,1136.22,20.25,22.80,23.65,24.25,24.45,24.45,24.60,24.75,1.513511,0.660716,2089364,1141.402,1145.68,9.46


In [7]:
output_df = spx_5day_df.filter(['SPX_tom', 'A_Delta'], axis=1)
output_df.head()

,SPX_tom,A_Delta
Date,,
2010-01-06,1141.69,4.55
2010-01-07,1144.98,3.29
2010-01-08,1146.98,2.00
2010-01-11,1136.22,-10.76
2010-01-12,1145.68,9.46


In [8]:
#Create list
#If "actual delta" > 0 (meaning tomorrow's price is higher than today's price)
#Assign "Buy" = 1, "Don't Buy" = 0
result_list = []
for i in range(0, len(output_df['A_Delta'])):
    if output_df['A_Delta'][i] > 0:
        result_list.append(1)
    else:
        result_list.append(0)

In [9]:
#Create Numpy array of list and prepare for "y-train" "y-test"

y_train_data = np.asarray(result_list[:1983])
y_test_data = np.asarray(result_list[1983:])

#Reshape array for later ML model
y_train = y_train_data.reshape(-1, 1)
y_test = y_test_data.reshape(-1,1)

print(y_train.shape)
print(y_test.shape)

(1983, 1)
(540, 1)


In [10]:
output_df['Act_Result'] = result_list
output_df.head()

,SPX_tom,A_Delta,Act_Result
Date,,,
2010-01-06,1141.69,4.55,1
2010-01-07,1144.98,3.29,1
2010-01-08,1146.98,2.00,1
2010-01-11,1136.22,-10.76,0
2010-01-12,1145.68,9.46,1


In [11]:
spx_5day_df = spx_5day_df.drop(columns=['stdev','SPX_tom', 'A_Delta'])

In [12]:
spx_5day_df.head()

,SPX,VIX_1,VIX_2,VIX_3,VIX_4,VIX_5,VIX_6,VIX_7,VIX_8,1/stdev,SPX_VOLUME,SPX_5day
Date,,,,,,,,,,,,
2010-01-06,1137.14,20.80,23.55,24.20,24.95,25.05,25.05,25.10,25.30,0.661645,1252015,1130.944
2010-01-07,1141.69,20.45,23.15,23.85,24.75,24.85,24.85,24.90,25.10,0.634749,1553963,1136.262
2010-01-08,1144.98,19.90,22.55,23.40,24.40,24.65,24.60,24.75,24.90,0.583323,1508175,1138.662
2010-01-11,1146.98,19.50,22.25,23.25,24.15,24.30,24.40,24.50,24.65,0.568027,1444997,1141.462
2010-01-12,1136.22,20.25,22.80,23.65,24.25,24.45,24.45,24.60,24.75,0.660716,2089364,1141.402


In [13]:
X_data = spx_5day_df.to_numpy()
print("X-Dim Total:")
print(X_data.shape[0])
print(X_data.shape[1])

X-Dim Total:
2523
12


In [14]:
train_df = spx_5day_df[:1983] 
test_df = spx_5day_df[1983:]

In [15]:

train_data = train_df.to_numpy()
print("x-train Dimensions:")
print(train_data.shape[0])
print(train_data.shape[1])
print("")

test_data = test_df.to_numpy()
print("x-test Dimensions:")
print(test_data.shape[0])
print(test_data.shape[1])
print("Note: Before Reshaping")

x-train Dimensions:
1983
12

x-test Dimensions:
540
12
Note: Before Reshaping


In [16]:
x_train = train_data.reshape(1, train_data.shape[0], train_data.shape[1])
print(x_train.shape)

(1, 1983, 12)


In [17]:
x_test = test_data.reshape(1, test_data.shape[0], test_data.shape[1])
print(x_test.shape)

(1, 540, 12)


In [18]:
from tensorflow.keras.models import Sequential
model = Sequential()

In [19]:
from tensorflow.keras.layers import LSTM

model.add(LSTM(32, input_shape=(1983, 12)))


W0926 20:05:13.643143 16856 deprecation.py:506] From C:\Users\blake\Anaconda3\lib\site-packages\tensorflow\python\ops\init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [20]:
from tensorflow.keras.layers import Dense, Activation
model.add(Dense(1))
model.add(Activation('sigmoid'))

In [21]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 32)                5760      
_________________________________________________________________
dense (Dense)                (None, 1)                 33        
_________________________________________________________________
activation (Activation)      (None, 1)                 0         
Total params: 5,793
Trainable params: 5,793
Non-trainable params: 0
_________________________________________________________________
None


In [22]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [23]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 32)                5760      
_________________________________________________________________
dense (Dense)                (None, 1)                 33        
_________________________________________________________________
activation (Activation)      (None, 1)                 0         
Total params: 5,793
Trainable params: 5,793
Non-trainable params: 0
_________________________________________________________________
None


In [24]:
# # Fit (train) the model
# model.fit(
#     x_train,
#     y_train_categorical,
#     epochs=1000,
#     shuffle=True,
#     verbose=2
# )

In [25]:
model.fit(x_train, y_train, shuffle=False, epochs=100, batch_size=1)

ValueError: Input arrays should have the same number of samples as target arrays. Found 1 input samples and 1983 target samples.

In [ ]:
# history = model.fit(X, y, batch_size=1, epochs=100, verbose=0)

In [ ]:
predictions = model.predict_classes(X, verbose=0)